In [ ]:
import numpy as np

# Normalize the data.
from sklearn import preprocessing
from numpy.linalg import cholesky, det, lstsq
from scipy.optimize import minimize
import scipy.spatial.distance as spdist

from sklearn.metrics import mean_squared_error
import scipy.io as spio
# Normalize the data.
from sklearn import preprocessing

def pass_arg(nsim, tr_size):

    # Compute the RMSE
    def root_mean_squared_error(y_true, y_pred):
        return np.sqrt(np.mean((y_pred-y_true)**2))
    
    print("tr_Size:",tr_size)
    
    #List of lakes to choose from
    lake = ['mendota' , 'mille_lacs']
    lake_num = 0  # 0 : mendota , 1 : mille_lacs
    lake_name = lake[lake_num]

    # Load features (Xc) and target values (Y)
    data_dir = '../../data/'
    filename = lake_name + '.mat'
    mat = spio.loadmat(data_dir + filename, squeeze_me=True,
    variable_names=['Y','Xc_doy','Modeled_temp'])
    Xc = mat['Xc_doy']
    Y = mat['Y']
        
    # train and test data
    trainX, trainY = Xc[:tr_size,:-1], Y[:tr_size]
    testX, testY = Xc[-50:,:-1], Y[-50:]
        
    def covSEard(hyp=None, x=None, z=None):
        ''' Squared Exponential covariance function with Automatic Relevance Detemination
         (ARD) distance measure. The covariance function is parameterized as:

         k(x^p,x^q) = sf2 * exp(-(x^p - x^q)' * inv(P) * (x^p - x^q)/2)

         where the P matrix is diagonal with ARD parameters ell_1^2,...,ell_D^2, where
         D is the dimension of the input space and sf2 is the signal variance.

         The hyperparameters are:

         hyp = [ log(ell_1)
                 log(ell_2)
                 ...
                 log(ell_D)
                 log(sqrt(sf2)) ]
        '''

#         [n, D] = x.shape
#         ell = 1/np.array(hyp[0:D])        # characteristic length scale
        
        
#         sf2 = np.array(hyp[D])**2         # signal variance
#         tmp = np.dot(np.diag(ell),x.T).T
#         A = spdist.cdist(np.dot(np.diag(ell),x.T).T, np.dot(np.diag(ell),z.T).T, 'sqeuclidean') # cross covariances

#         A = sf2*np.exp(-0.5*A)  

        [n, D] = x.shape
        ell = 1/np.array(hyp)[0,0:D]        # characteristic length scale
        
        sf2 = np.array(hyp)[0,D]**2         # signal variance
        A = spdist.cdist(np.dot(np.diag(ell),x.T).T, np.dot(np.diag(ell),z.T).T, 'sqeuclidean') # cross covariances
        A = sf2*np.exp(-0.5*A)
        return A


    def posterior_predictive(X_s, X_train, Y_train, *args):
        '''  
        Computes the suffifient statistics of the GP posterior predictive distribution 
        from m training data X_train and Y_train and n new inputs X_s.

        Args:
            X_s: New input locations (n x d).
            X_train: Training locations (m x d).
            Y_train: Training targets (m x 1).
            l: Kernel length parameter.
            sigma_f: Kernel vertical variation parameter.
            sigma_y: Noise parameter.

        Returns:
            Posterior mean vector (n x d) and covariance matrix (n x n).
        '''
        
        l = np.array(args)
        noise=l[-1]
        K = covSEard(hyp=[l[:-1]], x=X_train, z=X_train) + noise**2 * np.eye(len(X_train))
        K_s = covSEard(hyp=[l[:-1]], x=X_train, z=X_s)
        K_ss = covSEard(hyp=[l[:-1]], x=X_s, z=X_s)  + 1e-8 * np.eye(len(X_s))
#         K_inv = inv(K)
        K_inv = np.linalg.pinv(K)
    
        # Equation (4)
        mu_s = K_s.T.dot(K_inv).dot(Y_train)

        # Equation (5)
        cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
        
        return mu_s, cov_s


    def nll_fn(X_train, Y_train, noise=0, naive=False):
        '''
        Returns a function that computes the negative log marginal
        likelihood for training data X_train and Y_train and given 
        noise level.

        Args:
            X_train: training locations (m x d).
            Y_train: training targets (m x 1).
            noise: known noise level of Y_train.
            naive: if True use a naive implementation of Eq. (7), if 
                   False use a numerically more stable implementation. 

        Returns:
            Minimization objective.
        '''

        def nll_stable(theta):
            # Numerically more stable implementation of Eq. (7) as described
            # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
            # 2.2, Algorithm 2.1.
            K = covSEard(hyp=[theta], x=X_train, z=X_train) + \
            theta[-1]**2 * np.eye(len(X_train))
            
            K += 1e-6 * np.eye(*K.shape)
            L = cholesky(K)
            return np.sum(np.log(np.diagonal(L))) + \
                   0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train, rcond=None)[0], rcond=None)[0]) + \
                   0.5 * len(X_train) * np.log(2*np.pi)

        if naive:
            return nll_naive
        else:
            return nll_stable

    
    # Optimization
    res = minimize(nll_fn(trainX, trainY), x0 = 0.1*np.ones(trainX.shape[1]+2), 
                       bounds=((1e-5, None), (1e-5, None), (1e-5, None), (1e-5, None), (1e-5, None), (1e-5, None), (1e-5, None), (1e-5, None), (1e-5, None), (1e-5, None), (1e-5, None), (1e-5, None), (1e-5, None)),
                        method='L-BFGS-B')
        
#     res = minimize(nll_fn(trainX, trainY), x0 = [.1, .1, .1, 1e-3], 
#                    bounds=((1e-5, None), (1e-5, None), (1e-5, None), (1e-7, None)),
#                     method='L-BFGS-B')
    
#     print(f'After parameter optimization: l1={res.x[0]:.5f} l2={res.x[1]:.5f} sigma_f={res.x[2]:.5f}')
#     print(np.exp(res.x[0]),np.exp(res.x[1]), np.exp(res.x[2]))
    mu_s, cov_s = posterior_predictive(testX, trainX, trainY, *res.x)
    
    RMSE = []
    for ii in range(int(nsim)):
        samples = np.random.multivariate_normal(mu_s.ravel(), cov_s, 1).T
        rmse_test = np.sqrt(mean_squared_error(testY, samples))
        RMSE.append(rmse_test)
#         print("RMSE:", rmse_test)
        
#         RMSE.append(root_mean_squared_error(testY, samples))
#         print("RMSE:", root_mean_squared_error(testY, samples))

#     return samples, RMSE
    return RMSE

In [ ]:
mean_rmses=[]
std_rmses=[]
for ii in ([500,100,50]):
    test_rmse = pass_arg(10, ii)
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)
    mean_rmses.append(mean_rmse)
    std_rmses.append(std_rmse)

In [ ]:
mean_rmses

In [ ]:
std_rmses

In [82]:
import scipy.io as spio
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

tr_size = 500
use_YPhy = 0

#List of lakes to choose from
lake = ['mendota' , 'mille_lacs']
lake_num = 0  # 0 : mendota , 1 : mille_lacs
lake_name = lake[lake_num]

# Load features (Xc) and target values (Y)
data_dir = '../../data/'
filename = lake_name + '.mat'
mat = spio.loadmat(data_dir + filename, squeeze_me=True,
variable_names=['Y','Xc_doy','Modeled_temp'])
Xc = mat['Xc_doy']
Y = mat['Y']
Xc = Xc[:,:-1]
# scaler = preprocessing.StandardScaler()
# Xc = scaler.fit_transform(Xc)

# train and test data
trainX, testX, trainY, testY = train_test_split(Xc, Y, train_size=tr_size/Xc.shape[0], 
                                                test_size=tr_size/Xc.shape[0], random_state=42, shuffle=True)

# trainX, trainY = Xc[:tr_size,:-1], Y[:tr_size]
# testX, testY = Xc[tr_size:tr_size+50,:-1], Y[tr_size:tr_size+50]

In [83]:
import pandas as pd
pd.DataFrame(trainX)

,0,1,2,3,4,5,6,7,8,9,10
0,-1.019934,-0.580457,0.892478,0.154132,-1.315452,-1.206431,-0.267906,-0.902442,0.218041,-0.443125,-0.097724
1,1.871121,0.351973,-0.651273,-1.680445,-1.737498,-2.141416,3.732374,-0.913915,1.291422,-0.443125,2.302617
2,1.707195,-1.534375,1.453842,-1.438538,-1.688852,-2.101777,3.732374,1.512379,-0.088843,-0.443125,-0.097724
3,-0.453645,-0.110562,-0.510932,1.363849,-0.607454,-0.314231,-0.267906,0.101888,-0.560569,-0.290737,-0.097724
4,-0.036379,-0.193212,-0.089909,1.262509,0.062501,0.505631,-0.267906,-0.404804,0.843964,-0.392426,-0.097724
...,...,...,...,...,...,...,...,...,...,...,...
495,-0.155598,-0.706149,-1.633660,1.431189,0.276018,0.659003,-0.267906,-0.219960,-0.267175,-0.443125,-0.097724
496,-0.438743,-0.798475,-0.931955,-0.480360,-0.216351,-0.466446,-0.267906,-0.462452,-0.741821,-0.443125,-0.097724
497,0.380886,-1.063067,-1.493319,0.042995,1.007465,0.611271,-0.267906,1.007980,-0.356746,0.128508,-0.097724
498,0.157351,0.290198,0.050432,0.185339,0.489955,0.707243,-0.267906,-0.138474,-1.328273,-0.007148,-0.097724


In [84]:
pd.DataFrame(testX)

,0,1,2,3,4,5,6,7,8,9,10
0,1.126004,0.339802,-1.072296,-0.033589,-0.823050,-0.227416,-0.267906,-0.430686,0.285957,-0.443125,-0.097724
1,-0.021477,0.194635,1.453842,1.381620,1.204962,1.485757,-0.267906,0.302605,0.648515,-0.443125,-0.097724
2,-0.200305,-0.541765,-0.651273,0.272411,1.538305,1.005846,-0.267906,1.262587,-0.525829,-0.172396,-0.097724
3,-0.468548,0.194635,1.453842,0.934801,-0.773392,-0.709731,-0.267906,0.002175,0.271699,-0.281245,-0.097724
4,-0.230110,-0.541765,1.032819,1.379368,0.845192,0.964774,-0.267906,0.439747,-0.040784,0.828371,-0.097724
...,...,...,...,...,...,...,...,...,...,...,...
495,-0.677180,1.680927,-0.510932,1.252811,0.056269,0.694663,-0.267906,-1.198966,0.186237,-0.337671,-0.097724
496,-0.721887,-1.137394,-0.931955,1.034982,0.666376,0.719408,-0.267906,-0.315153,1.353656,-0.240476,-0.097724
497,0.246765,2.006363,1.032819,1.121993,-0.015915,0.555722,-0.267906,-1.619748,-0.004653,-0.443125,-0.097724
498,0.738543,-1.204883,-0.300421,0.079712,0.865881,0.774355,-0.267906,0.547244,-0.789519,-0.431670,-0.097724


In [142]:
import numpy as np
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process import GaussianProcessRegressor

kernel = C(5.0, (0.5, 1e1)) * RBF(length_scale = [1] * trainX.shape[1], length_scale_bounds=(1e-1, 1e7))
gp = GaussianProcessRegressor(kernel=kernel, alpha =1.5, n_restarts_optimizer=5)
gp.fit(trainX, trainY)
y_pred1, sigma1 = gp.predict(testX, return_std=True)

In [143]:
gp.kernel_

3.16**2 * RBF(length_scale=[0.933, 10.7, 1.11, 12.7, 4.34e+03, 4.4, 22.1, 4.12e+03, 16, 6.44e+03, 5.76e+04])

In [144]:
gp.score(trainX, trainY)

0.9586860920301515

In [145]:
gp.score(testX, testY)

0.9324753821297647

In [108]:
# y_predTr, sigma_tr = gp.predict(trainX, return_std=True)
# y_predTr

In [109]:
# gp.sample_y(trainX, n_samples=int(10)).shape

In [110]:
# y_pred1

In [113]:
# sigma1

In [ ]:
tr_size = 500

# train and test data
trainX, trainY = Xc[:tr_size,:-1], Y[:tr_size]
testX, testY = Xc[-50:,:-1], Y[-50:]
# kernel = C(1.0, (1e-6, 5)) * RBF(length_scale = [0.389, 0.144, 0.189, 2.94, 3.69, 2.34e+03, 1.63e+04, 3.95, 5.12, 5.23, 1.57e+03], length_scale_bounds=(1e-3, 1e5))
# kernel = C(10.0, (1e-1, 1e3)) * RBF(length_scale = [0.956, 182, 0.254, 27.6, 986, 28.2, 0.0224, 6.86, 6.27, 3.33, 36.1], length_scale_bounds=(1e-2, 1e5))
# kernel = C(1.0, (1e-1, 1e2)) * RBF(length_scale = [0.626, 3.29e+04, 0.0124, 2.95, 5.3, 1.63e+03, 1.42e+03, 4.94, 5.72, 17.7, 0.84], length_scale_bounds=(1e-2, 1e5))
# 6.91**2 ,   0.266, 109, 0.207, 5.45, 6.72, 1e+03, 1e+03, 11, 10.3, 19.7, 60.6
kernel = C(10.0, (1e-1, 1e3)) * RBF(length_scale = [1] * trainX.shape[1], length_scale_bounds=(1e-2, 1e3))

In [ ]:
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, copy_X_train=False)
gp.fit(trainX, trainY) #removing reshape results in a different error
y_pred, sigma = gp.predict(testX, return_std=True)

In [ ]:
gp.log_marginal_likelihood([6.91, 0.266, 109, 0.207, 5.45, 6.72, 1e+03, 1e+03, 11, 10.3, 19.7, 60.6])

In [ ]:
print(gp.kernel_.k1, ',',gp.kernel_.k2)

In [ ]:
y_pred

In [ ]:
sigma

In [ ]:
Xx = np.random.uniform(size=(1, 2))
ss, rmse = pass_arg(Xx, 100, 30)

In [ ]:
np.mean(rmse)

In [ ]:
Xx = np.random.uniform(size=(3, 2))
ss = pass_arg(Xx, 1, 30)
# print(ss)